In [23]:
import pandas as pd
import numpy as np
import plotly.express as px
import os

### Loading and Pivoting

In [21]:
# get the current working directory
base_path = os.getcwd() + '//Data//'

# Loading data
domestic_consumption = pd.read_csv(base_path + 'Coffee_domestic_consumption.csv')
coffee_production = pd.read_csv(base_path + 'Coffee_production.csv')
coffee_inventory = pd.read_csv(base_path + 'Coffee_green_coffee_inventorie.csv')

# Pivoting dataframes long
domestic_consumption = pd.melt(domestic_consumption, id_vars=['Country', 'Coffee type']).rename(columns = {'variable': 'Year',
                                                                                                           'value': 'Domestic Consumption'})
coffee_production = pd.melt(coffee_production, id_vars=['Country', 'Coffee type']).rename(columns = {'variable': 'Year',
                                                                                                           'value': 'Total Production'})

,Country,Coffee type,Year,Total Production
0,Angola,Robusta/Arabica,1990/91,3.000000e+06
1,Bolivia (Plurinational State of),Arabica,1990/91,7.380000e+06
2,Brazil,Arabica/Robusta,1990/91,1.637160e+09
3,Burundi,Arabica/Robusta,1990/91,2.922000e+07
4,Ecuador,Arabica/Robusta,1990/91,9.024000e+07
5,Indonesia,Robusta/Arabica,1990/91,4.464600e+08
6,Madagascar,Robusta,1990/91,5.892000e+07
7,Malawi,Arabica,1990/91,6.300000e+06
8,Papua New Guinea,Arabica/Robusta,1990/91,5.778000e+07
9,Paraguay,Arabica,1990/91,7.860000e+06


### Examining total Production + Consumption

In [32]:
grouped_consumption = domestic_consumption[['Country', 'Domestic Consumption']].groupby(['Country']).sum().reset_index()
grouped_production = coffee_production[['Country', 'Total Production']].groupby(['Country']).sum().reset_index()
grouped_consumption.head(10)
consumption_fig = px.bar(grouped_consumption, 
                         x = 'Country', y = 'Domestic Consumption', title = "Total Consumption by Country")
production_fig = px.bar(grouped_production.sort_values(by = 'Total Production', ascending= False), 
                         x = 'Country', y = 'Total Production', title = "Total Production by Country")
consumption_fig.show()
production_fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed